In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import tensorflow as tf
#use tensorflow version 1.x features
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

#separate feature and labels
features = columns[:-1]
label = columns[-1]

iris_train = pd.read_csv('iris_training.csv', header = None, names = columns)
iris_test = pd.read_csv('iris_test.csv', header = None, names = columns)

In [3]:
iris_train.sample(5)

,sepal_length,sepal_width,petal_length,petal_width,species
27,5.9,3.0,5.1,1.8,2
96,4.6,3.2,1.4,0.2,0
26,5.8,2.6,4.0,1.2,1
37,5.8,2.8,5.1,2.4,2
11,6.9,3.1,4.9,1.5,1


In [4]:
X_train = iris_train.iloc[:, :-1]

#one hot encode labels
y_train = iris_train.iloc[:, -1:]
y_train = pd.get_dummies(y_train.species)

X_test = iris_test.iloc[:, :-1]

#one hot encode labels
y_test = iris_test.iloc[:, -1:]
y_test = pd.get_dummies(y_test.species)

### 2. Create a simple softmax classifier with 3 neurons in output layer and no hidden layer using TensorFlow Core APIs. Save it as iris_softmax.py

Ref - 
- https://steadforce.com/first-steps-tensorflow-part-2/
- https://www.tensorflow.org/api_docs/python/tf/math/reduce_std
- https://www.kaggle.com/vijayshinva/iris-tensorflow-basic-softmax-regression


In [5]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(120, 4)
(120, 3)
(30, 4)
(30, 3)


In [6]:
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, 4])  #four features
y = tf.placeholder(tf.float32, shape=[None, 3])  #three classes

#placeholders for weights and biases
weight = tf.Variable(tf.zeros(shape=[4, 3]))
bias = tf.Variable(tf.zeros(shape=[3]))

#define softmax using tensorflow
y_predicted = tf.nn.softmax(tf.matmul(x, weight) + bias)

In [7]:
#define cost function
cross_entropy_cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_predicted), axis=1))

df = pd.DataFrame(columns=['Iteration','Learn Rate', 'Epoch', 'Mean Accuracy', 'Standard Deviation'])

#run whole thing 10 times
for i in range(1,11) :
    
    #try with different learn rate and iterations for gradient descent
    learn_rate = i/100
    epochs = i*1000
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learn_rate).minimize(cross_entropy_cost)
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs) :
        sess.run([optimizer], feed_dict={x:X_train, y:y_train})

    weight_hat, bias_hat = sess.run([weight, bias])
    prediction_accuracy = tf.equal(tf.argmax(y_predicted, 1), tf.argmax(y,1))
    
    mean_accuracy = sess.run(tf.reduce_mean(tf.cast(prediction_accuracy, tf.float32)),
                                            feed_dict={weight:weight_hat, bias:bias_hat, 
                                                                   x: X_test, y: y_test})
    std = sess.run(tf.math.reduce_std(tf.cast(prediction_accuracy, tf.float32)), feed_dict={weight:weight_hat, bias:bias_hat, 
                                                                        x: X_test, y: y_test})
    df = df.append({'Iteration': i, 'Learn Rate': learn_rate, 'Epoch': epochs,
                    'Mean Accuracy': mean_accuracy, 'Standard Deviation': std}, ignore_index=True)

df.head(10)

,Iteration,Learn Rate,Epoch,Mean Accuracy,Standard Deviation
0,1.0,0.01,1000.0,0.933333,0.249444
1,2.0,0.02,2000.0,0.966667,0.179505
2,3.0,0.03,3000.0,0.966667,0.179505
3,4.0,0.04,4000.0,0.966667,0.179505
4,5.0,0.05,5000.0,0.966667,0.179505
5,6.0,0.06,6000.0,0.966667,0.179505
6,7.0,0.07,7000.0,0.933333,0.249444
7,8.0,0.08,8000.0,0.933333,0.249444
8,9.0,0.09,9000.0,0.933333,0.249444
9,10.0,0.10,10000.0,0.933333,0.249444


### 3. Create an MLP classifier with 3 hidden layers of sizes 5, 10, 5 using tensorflow Core APIs. Save it as iris_mlp_tf.py

References :
- https://towardsdatascience.com/multi-layer-perceptron-using-tensorflow-9f3e218a4809
- https://becominghuman.ai/creating-your-own-neural-network-using-tensorflow-fa8ca7cc4d0e
- https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py
- https://www.javatpoint.com/multi-layer-perceptron-in-tensorflow
- https://steadforce.com/first-steps-tensorflow-part-3/

In [8]:
tf.reset_default_graph()
learning_rate = 0.01

graph = tf.Graph()

In [9]:
def mlp_model(x, n_h1, n_h2, n_h3):
    n_f = 4  #No of features
    n_labels = 3  #No of classes
    
    #hidden layer 1
    with tf.name_scope('hidden_layer1'):
        hidden_layer1_weight = tf.Variable(tf.truncated_normal([n_f, n_h1], 
                                                               mean=0, 
                                                               stddev=1/np.sqrt(n_f)), 
                                           name='hidden_layer1_weight')
        hidden_layer1_bias = tf.Variable(tf.zeros([n_h1]), name='hidden_layer1_bias')
        hidden_layer1 = tf.nn.relu(tf.matmul(x, hidden_layer1_weight) + hidden_layer1_bias)
  
    #hidden layer 2
    with tf.name_scope('hidden_layer2'):
        hidden_layer2_weight = tf.Variable(tf.truncated_normal([n_h1, n_h2],
                                                               mean=0, 
                                                               stddev=1/np.sqrt(n_h1)), 
                                           name='hidden_layer2_weight')
        hidden_layer2_bias = tf.Variable(tf.zeros([n_h2]), name='hidden_layer2_bias')
        hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer1, hidden_layer2_weight) + hidden_layer2_bias)
        
    #hidden layer 3
    with tf.name_scope('hidden_layer3'):
        hidden_layer3_weight = tf.Variable(tf.truncated_normal([n_h2, n_h3],
                                                               mean=0, 
                                                               stddev=1/np.sqrt(n_h2)), 
                                           name='hidden_layer3_weight')
        hidden_layer3_bias = tf.Variable(tf.zeros([n_h3]), name='hidden_layer3_bias')
        hidden_layer3 = tf.nn.relu(tf.matmul(hidden_layer2, hidden_layer3_weight) + hidden_layer3_bias)

    #output layer
    with tf.name_scope('output_layer'):
        output_layer_weight = tf.Variable(tf.truncated_normal([n_h3, n_labels],
                                                  mean=0, 
                                                  stddev=1/np.sqrt(n_h3)),
                                          name='output_layer_weight')
        output_layer_bias = tf.Variable(tf.zeros([3]), name='output_layer_bias')
        output_layer = tf.sigmoid(tf.matmul(hidden_layer3, output_layer_weight) + output_layer_bias)
    
    return output_layer 

In [15]:
y = tf.placeholder(tf.float32, [None, 3])
y_predicted = mlp_model(x, 5, 10, 5)

#cross entropy cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_predicted,
                                                                          labels=y))
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

correct_prediction = tf.equal(tf.argmax(y_train,1), tf.argmax(y_predicted,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epochs in range(1000):
        _, c= sess.run([optimizer,cost],feed_dict = {x: X_train, y: y_train})
        if(epochs + 1) % 100 == 0:
            print("Epoch:",epochs+1,"Cost:", c)
        print("Optimization Finished")

        test_result = sess.run(y_predicted,feed_dict = {x: X_train})
        correct_prediction = tf.equal(tf.argmax(test_result,1),tf.argmax(y_train,1))
        print("accuracy:", accuracy.eval({x: X_test, y: y_test}))

Optimization Finished


InvalidArgumentError: Incompatible shapes: [120] vs. [30]
	 [[node Equal_5 (defined at <ipython-input-15-f15755e05467>:12) ]]

Original stack trace for 'Equal_5':
  File "/Users/anmol/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/anmol/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/anmol/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/anmol/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/anmol/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/Users/anmol/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/Users/anmol/anaconda3/lib/python3.7/asyncio/base_events.py", li

In [17]:
X_test.shape
y_test.shape

(30, 3)